In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import sklearn
import time
import random
from scipy.stats import kurtosis,skew
from tqdm import tqdm

In [ ]:
Data=loadmat("/content/gdrive/MyDrive/Old_ADA_Bridge-damage_vibration_data.mat")

In [ ]:
class dataMatrix:
    def __init__(self,data):
        self.data=dict()
        self.classes=['time','a1','a2','a3','a4','a5','a6','a7','a8','o1','o2','o3']
        try:
            for i in range(12):
                self.data[self.classes[i]]=data[:8868,i] # 8868
        except:
            return
        

In [ ]:
class matData:
    def __init__(self,data):
        self.header='VBA'
        self.data=dict()
        for i in range(1,8):
            new_data=dict()
            for j in range(1,13,1):
                if data['VBA'][i][j].shape==(1,0):
                    continue
                else:
                    new_data['no'+str(j)]=dataMatrix(data['VBA'][i][j])
#                     print(new_data['no'+str(j)].data['time'].shape)
            self.data[data['VBA'][i][0][0]]=new_data
        

In [ ]:
dataObj=matData(Data)
classes=['time','a1','a2','a3','a4','a5','a6','a7','a8','o1','o2','o3']
case=['INT 30km','INT 50km','INT 40km','RCV 40km','DMG1 40km','DMG2 40km','DMG3 40km']

In [ ]:
x_train_ss=[]
y_train_ss=[]
for i in case[:4]:
    keys=dataObj.data[i].keys()
    for j in keys:
        for k in classes[1:-3]:
            x_train_ss.append(dataObj.data[i][j].data[k])
            y_train_ss.append(1)
            
for i in case[4:]:
    keys=dataObj.data[i].keys()
    for j in keys:
        all_sensor_data=[]
        for k in classes[1:-3]:
            x_train_ss.append(dataObj.data[i][j].data[k])
            y_train_ss.append(0)
           
X=abs(np.array(x_train_ss))
Y=np.array(y_train_ss)
print(X.shape)
print(Y.shape)

In [ ]:
X=X/np.max(X)
X=X*100.+.0000001
X=np.log(X)

In [ ]:
def shuffle(a,b):
  c=[]
  for i in range(len(a)):
    c.append([a[i],b[i]])
  # print(c)
  random.shuffle(c)
  # print(c)
  a=np.array([i[0] for i in c])
  b=np.array([i[1] for i in c])
  return a,b

In [ ]:
x_data,y_data=shuffle(X,Y)

In [ ]:
all_window_sizes=[1000,100,10,4096,2048,1024,512,256,128,64,32,16,8,4,2,1]
# all_window_sizes=[1]

In [ ]:
##### preparing single sensor data for multiple segments.
##### preparing new data mean,variance, std. ONLY FOR SINGLE SENSOR PREDICTION.
def segment_data(window_size,x):
  n_segments=8868//window_size
  mean_array=[]
  variance_array=[]
  std_array=[]
  skew_array=[]
  kurtosis_array=[]
  if window_size==1:
    dict_=dict()
    dict_['mean']=x
    dict_['var_']=x
    dict_['std_']=x
    dict_['skew']=x
    dict_['kurt']=x
    return dict_
  for i in tqdm(range(len(x_data))):
    mean=[]
    std=[]
    skew_=[]
    kurtosis_=[]
    for j in range(n_segments):
      idx=j*window_size
      #print(x[i][idx:idx+window_size].shape)
      mean.append(np.mean(x[i][idx:idx+window_size]))
      std.append(np.std(x[i][idx:idx+window_size]))
      skew_.append(skew(x[i][idx:idx+window_size],axis=0,bias=False))
      kurtosis_.append(kurtosis(x[i][idx:idx+window_size],axis=0,fisher=True,bias=False))

    variance=[j*j for j in std]

    mean_array.append(mean)
    variance_array.append(variance)
    std_array.append(std)
    skew_array.append(skew_)
    kurtosis_array.append(kurtosis_)

  dict_=dict()
  dict_['mean']=mean_array
  dict_['var_']=variance_array
  dict_['std_']=std_array
  dict_['skew']=skew_array
  dict_['kurt']=kurtosis_array

  return dict_

In [ ]:
# 100 segments 72.78, 71.50, 69.85, 96.32, 84
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

In [ ]:
def accuracy(array,y,split_size=70):
  n=len(y)
  train_size=(n*split_size)//100  # train_test_split 70:30
  model=SVC().fit(array[:train_size],y[:train_size]) # x as [length, window_size] y as [length,]
  train_acc=np.sum(model.predict(array[:train_size])==y[:train_size])/train_size
  acc=np.sum(model.predict(array[train_size:])==y[train_size:])/len(y[train_size:])
  return round(train_acc,3),round(acc,3)

In [ ]:
svm_log="/content/gdrive/MyDrive/project/logs/svm_logs4_with_abs_25jan_new_log_10_iterations_3.txt"
lr_log="/content/gdrive/MyDrive/project/logs/lr_logs_25jan_exp.txt"

In [ ]:
svm_log_data=dict()

In [ ]:
# with abs

with open(svm_log,"w") as log_file:
  for epoch in range(10):
    print('########################## epoch: ',epoch,' ###########################')
    for window_size in all_window_sizes:
      if window_size not in svm_log_data.keys():
        svm_log_data[window_size]=dict()
      x_data,y_data=shuffle(X,Y)
      segmented_data=segment_data(window_size,x_data)
      log_file.write("\n\nwindow_size: "+str(window_size))
      print("\nwindow_size: "+str(window_size))
      keys=list(segmented_data.keys())
      if window_size==1:
        keys=keys[:1]
      for key in keys:
        if key not in svm_log_data[window_size].keys():
          svm_log_data[window_size][key]=dict()
        for split_size in range(50,100,10):
          if split_size not in svm_log_data[window_size][key].keys():
            svm_log_data[window_size][key][split_size]=[]
          train_acc,acc=accuracy(segmented_data[key],y_data,split_size)
          svm_log_data[window_size][key][split_size].append(acc)
          arr=svm_log_data[window_size][key][split_size]
          average=round(sum(arr)/len(arr),3)
          text="\tsplit_size:"+str(split_size)+"\t "+str(key)+":  train--"+(str(train_acc)+'000')[:6]+" test--"+(str(acc)+'000')[:6]+" avg: "+(str(average)+'000')[:6]+" max: "+(str(max(arr))+'000')[:6]+" >>> "+str(arr)
          print(text)
          log_file.write(text)

########################## epoch:  0  ###########################


100%|██████████| 544/544 [00:02<00:00, 187.40it/s]



window_size: 1000
	split_size:50	 mean:  train--0.7170 test--0.6540 avg: 0.6540 max: 0.6540 >>> [0.654]
	split_size:60	 mean:  train--0.7150 test--0.6240 avg: 0.6240 max: 0.6240 >>> [0.624]
	split_size:70	 mean:  train--0.7080 test--0.6160 avg: 0.6160 max: 0.6160 >>> [0.616]
	split_size:80	 mean:  train--0.7060 test--0.6150 avg: 0.6150 max: 0.6150 >>> [0.615]
	split_size:90	 mean:  train--0.7080 test--0.6000 avg: 0.6000 max: 0.6000 >>> [0.6]
	split_size:50	 var_:  train--0.6360 test--0.5400 avg: 0.5400 max: 0.5400 >>> [0.54]
	split_size:60	 var_:  train--0.6200 test--0.5410 avg: 0.5410 max: 0.5410 >>> [0.541]
	split_size:70	 var_:  train--0.6180 test--0.5370 avg: 0.5370 max: 0.5370 >>> [0.537]
	split_size:80	 var_:  train--0.6210 test--0.4950 avg: 0.4950 max: 0.4950 >>> [0.495]
	split_size:90	 var_:  train--0.6180 test--0.4730 avg: 0.4730 max: 0.4730 >>> [0.473]
	split_size:50	 std_:  train--0.6650 test--0.6100 avg: 0.6100 max: 0.6100 >>> [0.61]
	split_size:60	 std_:  train--0.6560 te

100%|██████████| 544/544 [00:33<00:00, 16.04it/s]



window_size: 100
	split_size:50	 mean:  train--0.7170 test--0.6990 avg: 0.6990 max: 0.6990 >>> [0.699]
	split_size:60	 mean:  train--0.7090 test--0.7060 avg: 0.7060 max: 0.7060 >>> [0.706]
	split_size:70	 mean:  train--0.7050 test--0.6770 avg: 0.6770 max: 0.6770 >>> [0.677]
	split_size:80	 mean:  train--0.7010 test--0.6970 avg: 0.6970 max: 0.6970 >>> [0.697]
	split_size:90	 mean:  train--0.7080 test--0.7090 avg: 0.7090 max: 0.7090 >>> [0.709]
	split_size:50	 var_:  train--0.6910 test--0.6320 avg: 0.6320 max: 0.6320 >>> [0.632]
	split_size:60	 var_:  train--0.6930 test--0.6330 avg: 0.6330 max: 0.6330 >>> [0.633]
	split_size:70	 var_:  train--0.7130 test--0.5610 avg: 0.5610 max: 0.5610 >>> [0.561]
	split_size:80	 var_:  train--0.6850 test--0.5600 avg: 0.5600 max: 0.5600 >>> [0.56]
	split_size:90	 var_:  train--0.6950 test--0.5640 avg: 0.5640 max: 0.5640 >>> [0.564]
	split_size:50	 std_:  train--0.6990 test--0.6800 avg: 0.6800 max: 0.6800 >>> [0.68]
	split_size:60	 std_:  train--0.7090 t

...

100%|██████████| 544/544 [00:27<00:00, 20.00it/s]



window_size: 128
	split_size:50	 mean:  train--0.7020 test--0.7060 avg: 0.6780 max: 0.7100 >>> [0.706, 0.64, 0.673, 0.654, 0.71, 0.654, 0.706]
	split_size:60	 mean:  train--0.7180 test--0.7390 avg: 0.6900 max: 0.7390 >>> [0.688, 0.647, 0.683, 0.674, 0.706, 0.693, 0.739]
	split_size:70	 mean:  train--0.7050 test--0.7200 avg: 0.7010 max: 0.7200 >>> [0.701, 0.683, 0.701, 0.713, 0.701, 0.689, 0.72]
	split_size:80	 mean:  train--0.7010 test--0.7520 avg: 0.7180 max: 0.7520 >>> [0.697, 0.725, 0.716, 0.734, 0.697, 0.706, 0.752]
	split_size:90	 mean:  train--0.7080 test--0.7090 avg: 0.7070 max: 0.7820 >>> [0.691, 0.782, 0.727, 0.709, 0.673, 0.655, 0.709]
	split_size:50	 var_:  train--0.6690 test--0.6360 avg: 0.6170 max: 0.6470 >>> [0.647, 0.585, 0.621, 0.596, 0.629, 0.603, 0.636]
	split_size:60	 var_:  train--0.6630 test--0.6420 avg: 0.6190 max: 0.6420 >>> [0.619, 0.61, 0.628, 0.592, 0.601, 0.638, 0.642]
	split_size:70	 var_:  train--0.6760 test--0.6650 avg: 0.6230 max: 0.6830 >>> [0.579, 0.57

100%|██████████| 544/544 [00:53<00:00, 10.19it/s]



window_size: 64
	split_size:50	 mean:  train--0.7430 test--0.6800 avg: 0.6720 max: 0.6950 >>> [0.662, 0.695, 0.621, 0.688, 0.688, 0.669, 0.68]
	split_size:60	 mean:  train--0.7330 test--0.6650 avg: 0.6760 max: 0.7200 >>> [0.683, 0.674, 0.642, 0.72, 0.665, 0.683, 0.665]
	split_size:70	 mean:  train--0.7240 test--0.6830 avg: 0.6890 max: 0.7320 >>> [0.689, 0.689, 0.634, 0.732, 0.683, 0.713, 0.683]
	split_size:80	 mean:  train--0.7310 test--0.6790 avg: 0.6690 max: 0.7160 >>> [0.688, 0.624, 0.606, 0.706, 0.661, 0.716, 0.679]
	split_size:90	 mean:  train--0.7220 test--0.7090 avg: 0.6750 max: 0.7640 >>> [0.764, 0.655, 0.564, 0.727, 0.582, 0.727, 0.709]
	split_size:50	 var_:  train--0.7350 test--0.5920 avg: 0.6090 max: 0.6320 >>> [0.61, 0.629, 0.585, 0.592, 0.625, 0.632, 0.592]
	split_size:60	 var_:  train--0.7020 test--0.6010 avg: 0.6000 max: 0.6380 >>> [0.569, 0.61, 0.573, 0.615, 0.596, 0.638, 0.601]
	split_size:70	 var_:  train--0.7450 test--0.6160 avg: 0.6110 max: 0.6650 >>> [0.561, 0.634

 31%|███▏      | 170/544 [00:33<01:15,  4.95it/s]

In [ ]:
def lr_accuracy(x,y,split):
  train_size=(len(y)*split)//100
  x_train=x[:train_size]
  y_train=y[:train_size]
  x_test=x[train_size:]
  y_test=y[train_size:]
  linear_model=LinearRegression().fit(x_train,y_train)
  pred=linear_model.predict(x_test)
  pred_norm=(pred-np.min(pred))/(np.max(pred)-np.min(pred))
  threshold=[]
  acc=[]
  acc_=0
  acc__=0
  thresh=.1
  thresh_=0
  while thresh<.9:
    acc_=np.sum((y_test==(pred_norm>thresh).astype(np.int_)).astype(np.int_))/len(y_test)
    acc.append(acc_)
    if acc_>acc__:
      acc__=acc_
      thresh_=thresh
    threshold.append(thresh)
    thresh+=0.02
  start_time=time.time()
  _=linear_model.predict([x_test[-1]])
  end_time=time.time()-start_time
  return acc__

In [ ]:
# with abs

with open(lr_log,"w") as log_file:
  for window_size in all_window_sizes:
    x_data,y_data=shuffle(X,Y)
    segmented_data=segment_data(window_size,x_data)
    log_file.write("\n\nwindow_size: "+str(window_size))
    print("\nwindow_size: "+str(window_size))
    for key in segmented_data.keys():
      for split_size in range(50,100,10):
        acc=lr_accuracy(np.array(segmented_data[key]),y_data,split_size)
        text="\tsplit_size:"+str(split_size)+"\t "+str(key)+": test--"+str(acc)
        print(text)
        log_file.write(text)

100%|██████████| 544/544 [00:02<00:00, 190.10it/s]



window_size: 1000
	split_size:50	 mean: test--0.75
	split_size:60	 mean: test--0.7935779816513762
	split_size:70	 mean: test--0.8109756097560976
	split_size:80	 mean: test--0.7889908256880734
	split_size:90	 mean: test--0.8181818181818182
	split_size:50	 variance: test--0.6617647058823529
	split_size:60	 variance: test--0.6743119266055045
	split_size:70	 variance: test--0.6951219512195121
	split_size:80	 variance: test--0.6788990825688074
	split_size:90	 variance: test--0.7454545454545455
	split_size:50	 std: test--0.6838235294117647
	split_size:60	 std: test--0.6880733944954128
	split_size:70	 std: test--0.7195121951219512
	split_size:80	 std: test--0.7339449541284404
	split_size:90	 std: test--0.7090909090909091
	split_size:50	 skew: test--0.6102941176470589
	split_size:60	 skew: test--0.5871559633027523
	split_size:70	 skew: test--0.5853658536585366
	split_size:80	 skew: test--0.5688073394495413
	split_size:90	 skew: test--0.6181818181818182
	split_size:50	 kurtosis: test--0.577205

100%|██████████| 544/544 [00:36<00:00, 14.79it/s]



window_size: 100
	split_size:50	 mean: test--0.7316176470588235
	split_size:60	 mean: test--0.7477064220183486
	split_size:70	 mean: test--0.7439024390243902
	split_size:80	 mean: test--0.7431192660550459
	split_size:90	 mean: test--0.7818181818181819
	split_size:50	 variance: test--0.7536764705882353
	split_size:60	 variance: test--0.7339449541284404
	split_size:70	 variance: test--0.7134146341463414
	split_size:80	 variance: test--0.6972477064220184
	split_size:90	 variance: test--0.6545454545454545
	split_size:50	 std: test--0.7352941176470589
	split_size:60	 std: test--0.7339449541284404
	split_size:70	 std: test--0.7195121951219512
	split_size:80	 std: test--0.7155963302752294
	split_size:90	 std: test--0.6909090909090909
	split_size:50	 skew: test--0.6176470588235294
	split_size:60	 skew: test--0.6055045871559633
	split_size:70	 skew: test--0.5975609756097561
	split_size:80	 skew: test--0.6238532110091743
	split_size:90	 skew: test--0.6909090909090909
	split_size:50	 kurtosis: t

100%|██████████| 544/544 [05:33<00:00,  1.63it/s]


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


	split_size:70	 kurtosis: test--0.4878048780487805
	split_size:80	 kurtosis: test--0.5137614678899083


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


	split_size:90	 kurtosis: test--0.5818181818181818


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
